# Задача

Заказчику, крупной телеком компании «Ниединогоразрыва.ком», нужна модель для прогноза оттока клиентов.

# Цель

Модель будет оцениваться по метрике ROC-AUC, оценка модель будет производиться по следующим показателям:

0 sp - AUC-ROC < 0.75
4 sp - 0.75 ≤ AUC-ROC < 0.81
4.5 sp - 0.81 ≤ AUC-ROC < 0.85
5 sp - 0.85 ≤ AUC-ROC < 0.87
5.5 sp - 0.87 ≤ AUC-ROC < 0.88
6 sp - AUC-ROC ≥ 0.88


# Описание данных

 - BeginDate – дата начала пользования услугами,
 - EndDate – дата окончания пользования услугами,
 - Type – тип оплаты: ежемесячный, годовой и тд,
 - PaperlessBilling – электронный документ об оплате,
 - PaymentMethod – способ оплаты,
 - MonthlyCharges – ежемесячные траты на услуги,
 - TotalCharges – всего потрачено денег на услуги
 - Dependents – наличие иждивенцев
 - Senior Citizen – наличие пенсионного статуса по возрасту
 - Partner – наличие супруга(и)
 - MultipleLines – наличие возможности ведения параллельных линий во время звонка

# Доп. требования

Параметр random_state имеет фиксированное значение 250722

# Шаг 1. Загрузка и анализ данных

Загрузим необходимые библиотеки

In [166]:
from datetime import datetime

import numpy as np
import pandas as pd

import plotly.express as px

from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.utils import shuffle

from catboost import CatBoostClassifier

# фиксируем random_state
rs = 250722

Загрузим данные и посмотрим на данные

In [167]:
try:
	contract = pd.read_csv('../data/contract.csv')
	internet = pd.read_csv('../data/internet.csv')
	personal = pd.read_csv('../data/personal.csv')
	phone = pd.read_csv('../data/phone.csv')
except:
	contract = pd.read_csv('/datasets/final_provider/contract.csv')
	internet = pd.read_csv('/datasets/final_provider/internet.csv')
	personal = pd.read_csv('/datasets/final_provider/personal.csv')
	phone = pd.read_csv('/datasets/final_provider/phone.csv')

# зададим имена датасетам и создадим список с табличками для удобства
contract.name = 'contract_data'
internet.name = 'internet_data'
personal.name = 'personal_data'
phone.name = 'phone_data'

data_list = [contract, internet, personal, phone]

In [168]:
for data in data_list:
	print(f"{'=' * 30} {data.name} {'='*30}")
	data.info()

============================== contract_data ==============================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
============================== internet_data ==============================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID     


В таблицах отсутствуют пропуски.

In [169]:
for data in data_list:
	display(f"{'=' * 30} {data.name} {'='*30}")
	display(data.sample(10))

'============================== contract_data =============================='

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
3989,7634-WSWDB,2019-06-01,No,Month-to-month,Yes,Electronic check,38.50,330.8
3289,9626-VFRGG,2016-09-01,No,Month-to-month,Yes,Bank transfer (automatic),88.50,3645.05
3551,7594-RQHXR,2019-09-01,2019-10-01 00:00:00,Month-to-month,Yes,Electronic check,79.60,79.6
3893,5816-SCGFC,2019-07-01,No,Month-to-month,Yes,Mailed check,51.30,419.35
6648,2164-SOQXL,2018-06-01,No,Month-to-month,No,Mailed check,20.05,406.05
2926,0106-UGRDO,2014-05-01,No,Two year,Yes,Electronic check,116.00,8182.85
5135,1156-ZFYDO,2020-01-01,No,Month-to-month,No,Electronic check,19.75,19.75
2754,6351-SCJKT,2019-11-01,No,Month-to-month,No,Mailed check,41.35,107.25
2748,2495-INZWQ,2019-08-01,2019-12-01 00:00:00,Month-to-month,Yes,Electronic check,44.55,174.3
1553,7776-QGYJC,2016-11-01,No,Month-to-month,Yes,Bank transfer (automatic),81.50,3107.3


'============================== internet_data =============================='

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
4141,8745-PVESG,DSL,Yes,No,Yes,Yes,No,No
4489,9730-DRTMJ,DSL,Yes,No,No,Yes,Yes,No
660,0295-QVKPB,DSL,No,No,Yes,Yes,Yes,No
3770,0584-BJQGZ,DSL,Yes,Yes,Yes,Yes,Yes,No
1400,7198-GLXTC,Fiber optic,No,No,No,No,Yes,No
1047,3372-KWFBM,Fiber optic,No,No,No,No,Yes,No
3468,3716-LRGXK,Fiber optic,No,No,Yes,No,Yes,Yes
4220,8124-NZVGJ,DSL,No,No,No,Yes,Yes,Yes
5209,3588-WSTTJ,Fiber optic,Yes,No,No,No,Yes,Yes
3801,6848-YLDFR,DSL,No,Yes,No,Yes,Yes,Yes


'============================== personal_data =============================='

,customerID,gender,SeniorCitizen,Partner,Dependents
2909,9619-GSATL,Female,0,No,No
1994,8022-BECSI,Male,0,No,No
6610,9696-RMYBA,Male,0,No,No
6744,6500-JVEGC,Male,0,No,No
4782,6175-IRFIT,Male,0,No,No
1151,8992-CEUEN,Female,0,No,No
3608,3208-YPIOE,Male,0,No,No
4633,1820-DJFPH,Female,0,Yes,Yes
6661,3704-IEAXF,Female,0,Yes,Yes
4032,4098-NAUKP,Male,1,Yes,Yes


'============================== phone_data =============================='

,customerID,MultipleLines
3068,0396-HUJBP,No
809,8975-SKGRX,Yes
3884,5811-IWXYM,Yes
5909,1763-KUAAW,No
3542,3886-CERTZ,Yes
238,4385-GZQXV,No
3062,2516-VQRRV,Yes
642,1029-QFBEN,No
3970,2718-YSKCS,No
6209,5146-YYFRZ,No


В данных присутствует большое количество категориальных признаков, две колонки с вещественными признаками (MonthlyCharges и TotalCharges)

In [170]:
for data in data_list:
	display(f"{'=' * 30} {data.name} {'='*30}")
	display(f'Количество строк в таблице: {data.shape[0]}')
	display(f'Количество уникальных клиентов: {len(data["customerID"].unique())}')

'============================== contract_data =============================='

'Количество строк в таблице: 7043'

'Количество уникальных клиентов: 7043'

'============================== internet_data =============================='

'Количество строк в таблице: 5517'

...

'Количество строк в таблице: 7043'

'Количество уникальных клиентов: 7043'

'============================== phone_data =============================='

'Количество строк в таблице: 6361'

'Количество уникальных клиентов: 6361'

Каждая запись в таблице содержит уникального клиента. Но количество записей в таблицах разное, значит после джоина таблиц по customerID у нас появятся пропуски в данных.

Попробуем это сделать и посмотрим на количество пропусков.
Для этого установим поле customerID в качестве индекса и сджоиним таблицы по индексу.

In [171]:
contract = contract.set_index('customerID')
personal = personal.set_index('customerID')
phone = phone.set_index('customerID')
internet = internet.set_index('customerID')

full_data = contract.join([personal, phone, internet])
full_data.name = 'full_data'

In [172]:
def get_missing_values(data: pd.DataFrame) -> None:
	"""
	Выводит данные о пропусках в колонках по датафрейму.
	Не изменяет данные внутри датафрейма.

	:param data: pd.DataFrame
	:return: None
	"""
	# получаем имена колонок датафрейма
	columns = data.columns.to_list()
	data_len = len(data)
	# объявляем счетчик
	counter = -1
	print('='*60)
	# если есть пропуски в данных - выводим информацию о пропусках по колонкам
	if sum(data.isnull().sum()) > 0:
		print(f'Количество записей в датафрейме {data.name}: {data_len} \n')
		print(f'В датафрейме {data.name} имеются следующие пропуски:')
		for i in data.isnull().sum():
			counter += 1
			if i > 0:
				print(f'  - в колонке {columns[counter]}: {i} пропусков, это {i/data_len:0.2%} об общего объема данных')
	else:
		print(f'Отлично, в датафрейме {data.name} отсутствуют пропуски.')

# посмотрим на пропуски в данных
get_missing_values(full_data)

Количество записей в датафрейме full_data: 7043 

В датафрейме full_data имеются следующие пропуски:
  - в колонке MultipleLines: 682 пропусков, это 9.68% об общего объема данных
  - в колонке InternetService: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке OnlineSecurity: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке OnlineBackup: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке DeviceProtection: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке TechSupport: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке StreamingTV: 1526 пропусков, это 21.67% об общего объема данных
  - в колонке StreamingMovies: 1526 пропусков, это 21.67% об общего объема данных


In [173]:
full_data.head()

,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
customerID,,,,,,,,,,,,,,,,,,,
7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,NaN,DSL,No,Yes,No,No,No,No
5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,No,DSL,Yes,No,Yes,No,No,No
3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,No,DSL,Yes,Yes,No,No,No,No
7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,NaN,DSL,Yes,No,Yes,Yes,No,No
9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,No,Fiber optic,No,No,No,No,No,No


Посмотрим на типы данных

In [174]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BeginDate         7043 non-null   object 
 1   EndDate           7043 non-null   object 
 2   Type              7043 non-null   object 
 3   PaperlessBilling  7043 non-null   object 
 4   PaymentMethod     7043 non-null   object 
 5   MonthlyCharges    7043 non-null   float64
 6   TotalCharges      7043 non-null   object 
 7   gender            7043 non-null   object 
 8   SeniorCitizen     7043 non-null   int64  
 9   Partner           7043 non-null   object 
 10  Dependents        7043 non-null   object 
 11  MultipleLines     6361 non-null   object 
 12  InternetService   5517 non-null   object 
 13  OnlineSecurity    5517 non-null   object 
 14  OnlineBackup      5517 non-null   object 
 15  DeviceProtection  5517 non-null   object 
 16  TechSupport       5517 non-null 

TotalCharges явно не соответствует типу object, посмотрим что там за знчения и конвертируем его во float

In [175]:
full_data['TotalCharges'].value_counts().head()

         11
20.2     11
19.75     9
20.05     8
19.9      8
Name: TotalCharges, dtype: int64

Видим, что в столбце присутствуют пустые строки, посмотрим что это за клиенты

In [176]:
full_data[full_data['TotalCharges'] == ' ']

,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
customerID,,,,,,,,,,,,,,,,,,,
4472-LVYGI,2020-02-01,No,Two year,Yes,Bank transfer (automatic),52.55,,Female,0,Yes,Yes,NaN,DSL,Yes,No,Yes,Yes,Yes,No
3115-CZMZD,2020-02-01,No,Two year,No,Mailed check,20.25,,Male,0,No,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5709-LVOEQ,2020-02-01,No,Two year,No,Mailed check,80.85,,Female,0,Yes,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes
4367-NUYAO,2020-02-01,No,Two year,No,Mailed check,25.75,,Male,0,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1371-DWPAZ,2020-02-01,No,Two year,No,Credit card (automatic),56.05,,Female,0,Yes,Yes,NaN,DSL,Yes,Yes,Yes,Yes,Yes,No
7644-OMVMY,2020-02-01,No,Two year,No,Mailed check,19.85,,Male,0,Yes,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3213-VVOLG,2020-02-01,No,Two year,No,Mailed check,25.35,,Male,0,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2520-SGTTA,2020-02-01,No,Two year,No,Mailed check,20.00,,Female,0,Yes,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2923-ARZLG,2020-02-01,No,One year,Yes,Mailed check,19.70,,Male,0,Yes,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Это новые клиенты, которые еще не успели произвести ни одной транзакции. Можем удалить этих ребят из выборки.

In [177]:
index_to_drop = full_data[full_data['TotalCharges'] == ' '].index
full_data = full_data.drop(index_to_drop)
full_data['TotalCharges'] = full_data['TotalCharges'].astype(float)

In [178]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BeginDate         7032 non-null   object 
 1   EndDate           7032 non-null   object 
 2   Type              7032 non-null   object 
 3   PaperlessBilling  7032 non-null   object 
 4   PaymentMethod     7032 non-null   object 
 5   MonthlyCharges    7032 non-null   float64
 6   TotalCharges      7032 non-null   float64
 7   gender            7032 non-null   object 
 8   SeniorCitizen     7032 non-null   int64  
 9   Partner           7032 non-null   object 
 10  Dependents        7032 non-null   object 
 11  MultipleLines     6352 non-null   object 
 12  InternetService   5512 non-null   object 
 13  OnlineSecurity    5512 non-null   object 
 14  OnlineBackup      5512 non-null   object 
 15  DeviceProtection  5512 non-null   object 
 16  TechSupport       5512 non-null 

В данных получилось довольно большое количество пропусков и мы не можем от них просто так избавиться.
Все пропуски присутствуют в полях с категориальными данными, так что мы можем попробовать заполнить их разными способами, а именно:
 - ввести новую категорию 'unknown' для данных с пропусками
 - заполнить пропуски самым частотным значением признака
 - посчитать доли для значений признака с пропусками, и в соответствующих значениям долях заполнить пропуски (например если у нас в признаке только значения yes или no и они распределены в долях 70% yes и 30% no, то все пропуски это 100% и из них мы рандомно заполняем 70% пропусков значением yes и 30% пропусков значением no)

Также нам необходимо создать целевой признак. Брать данные для генерации целевого признака будем из колонки EndDate таблицы contract, если no = 0, а если есть дата, то 1.
Сделаем это.


In [179]:
full_data['churn'] = full_data['EndDate'].apply(lambda x: 0 if x == 'No' else 1)

Посмотрим на распределение целевого признака.

In [180]:
full_data['churn'].value_counts(normalize=True)

0    0.734215
1    0.265785
Name: churn, dtype: float64

~26.5% клиентов покинули

Посмотрим на корреляцию с целевым признаком.

In [181]:
cat_features = ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines', 'InternetService' , 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

encoder = LabelEncoder()

for feature in cat_features:
	full_data[feature] = encoder.fit_transform(full_data[feature].fillna('unknown'))

In [182]:
px.imshow(full_data.corr(), text_auto=True).show()

Видим, что целевой признак не имеет сильно скоррелированных признаков, зато признаки из таблицы internet довольно сильно скоррелированы между собой, может быть имеет смысл избавиться от этих признаков или сжать их с помощью PCA.

## Вопросы:

Что за колонки Partner и Dependents в таблице personal?

## План дальнейших действий:

 - заполнить пропуски в категориальных признаках
 - сжать данные высокоскоррелированных признаков методом РСА
 - произвести апсемплинг низкочастотного целевого признака
 - подобрать гиперпараметры и обучить разные модели классификации
 - подобрать threshold для моделей
 - сравнить метрики полученных моделей, построить conflusion matrix
 - написать выводы

Сожмем высокоскоррелированные признаки методом PCA

In [183]:
high_corr_features = ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
pca = PCA(n_components=1, random_state=rs)

full_data['pca_feature'] = pca.fit_transform(full_data[high_corr_features])

# удалим сжатые признаки
full_data = full_data.drop(high_corr_features, axis=1)

full_data.sample(5)

,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,MultipleLines,churn,pca_feature
customerID,,,,,,,,,,,,,,
0925-VYDLG,2019-09-01,2019-12-01 00:00:00,0,0,2,75.25,242.00,0,0,0,0,1,1,-1.741861
9818-XQCUV,2019-12-01,No,0,0,3,20.35,45.30,0,0,0,0,0,0,3.212431
3374-LXDEV,2018-09-01,2019-10-01 00:00:00,0,0,2,89.40,1132.35,0,0,0,0,1,1,-0.576232
8709-KRDVL,2017-05-01,No,0,1,2,100.00,3320.60,0,0,0,0,1,0,-0.603215
0228-MAUWC,2018-07-01,No,0,0,2,59.55,1144.60,1,0,0,0,1,0,-1.286681


Избавимся от дисбаланса классов с помощью апсемплинга

In [184]:
def up_sample(
		features: pd.DataFrame,
		target: pd.DataFrame,
		repeat: int=0,
		repeat_auto: bool=False,
		zeros: bool=True,
		random_state: int = 42) -> tuple[pd.DataFrame, pd.DataFrame]:
	"""

	:param features: Features data
	:param target: Target data
	:param repeat: Repeat count target features
	:param repeat_auto: Automatic estimate repeat count
	:param zeros: Feature to repeat. If True - repeat zeros, else ones.
	:return: upsampled features and target
	"""

	features_zeros = features[target == 0]
	features_ones = features[target == 1]
	target_zeros = target[target == 0]
	target_ones = target[target == 1]

	# автоматическое сэмплирование до равных размеров features и target
	if repeat_auto:
		if zeros:
			repeat_features = np.round(len(features_ones) / len(features_zeros)).astype('int')
			repeat_target = np.round(len(target_ones) / len(target_zeros)).astype('int')
			features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat_features)
			target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat_target)

			features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=random_state)

			return features_upsampled, target_upsampled
		else:
			repeat_features = np.round(len(features_zeros) / len(features_ones)).astype('int')
			repeat_target = np.round(len(target_zeros) / len(target_ones)).astype('int')
			features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat_features)
			target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat_target)

			features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=random_state)

			return features_upsampled, target_upsampled

	# ручное сэмплирование, если указано количество повторений
	else:
		features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
		target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

	features_upsampled, target_upsampled = shuffle(
		features_upsampled, target_upsampled, random_state=random_state)

	return features_upsampled, target_upsampled

In [185]:
# удалим ненужные даты и таргет из признаков
features = full_data.drop(['BeginDate', 'EndDate', 'churn'], axis=1)
target = full_data['churn']

features, target = up_sample(features, target, repeat_auto=True, zeros=False, random_state=rs)

# Шаг 3. Обучим модель

In [186]:
# отделим таргет от трейна
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.2, random_state=rs)

## Шаг 3.1 Попробуем решить задачу с помощью CatboostClassifier.

Сперва отберем наиболее важные признаки для Catboost.

In [187]:
cat_features = ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines']
cbc = CatBoostClassifier(cat_features=cat_features, task_type='GPU', silent=True, random_state=rs)
cbc.fit(features_train, target_train)

Посмотрим на важность признаков

In [188]:
feat_importances = pd.Series(cbc.feature_importances_, index=features_train.columns).sort_values(ascending=False)
fig = px.bar(feat_importances, title='Наиболее значимые признаки для линейной регрессии')

fig.update_layout(
    showlegend=False
)

fig.update_xaxes(
    title='Признак'
)

fig.update_yaxes(
    title='Коэфициент важности'
)

fig.show()

Признаки Partner, SeniorCitizen и gender имеют низкую важность для классификации модели, но пока попробуем обучить модель с ними, т.к. они имеют ненулевую важность.
Также видим, что наш сгенерированный с помощью PCA метод имеет хороший вес для модели.

In [189]:
def plot_roc_auc_curve(target_valid: pd.DataFrame, probabilities_one_valid: pd.DataFrame) -> None:
	"""
	Plotting roc_auc curve function

	:param target_valid:
	:param probabilities_one_valid:
	:return:
	"""

	fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)

	# линия предсказания модели
	fig = px.area(
		x=fpr, y=tpr,
		title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
		labels=dict(x='False Positive Rate', y='True Positive Rate'),
		width=700, height=500
	)
	# линия предсказания случайной модели
	fig.add_shape(
		type='line', line=dict(dash='dash'),
		x0=0, x1=1, y0=0, y1=1
	)

	fig.show()

In [190]:
probabilities_valid = cbc.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

plot_roc_auc_curve(target_test, probabilities_one_valid)

In [192]:
# удалим признаки с низкой важностью
new_features = features_train.drop(['Partner', 'Dependents', 'gender'], axis=1)
new_cat_features = ['Type', 'PaperlessBilling', 'PaymentMethod', 'MultipleLines']
cbc = CatBoostClassifier(cat_features=new_cat_features, task_type='GPU', silent=True, random_state=rs)
cbc.fit(new_features, target_train)

In [193]:
feat_importances = pd.Series(cbc.feature_importances_, index=new_features.columns).sort_values(ascending=False)
fig = px.bar(feat_importances, title='Наиболее значимые признаки для линейной регрессии')

fig.update_layout(
    showlegend=False
)

fig.update_xaxes(
    title='Признак'
)

fig.update_yaxes(
    title='Коэфициент важности'
)

fig.show()

In [195]:
probabilities_valid = cbc.predict_proba(features_test.drop(['Partner', 'Dependents', 'gender'], axis=1))
probabilities_one_valid = probabilities_valid[:, 1]

plot_roc_auc_curve(target_test, probabilities_one_valid)

Удаление признаков чуть ухудшило метрику.

In [197]:
confusion_matrix(target_test, cbc.predict(features_test.drop(['Partner', 'Dependents', 'gender'], axis=1)))

array([[721, 320],
       [158, 955]], dtype=int64)

## Шаг 3.2 Попробуем решить задачу с помощью RandomFeorest

Подберем гиперпараметры для модели

In [213]:
params = {
	'n_estimators': [30, 50, 70, 100],
	'max_depth': [3, 5, 7, 10, 12, 13, 14, 16],
	'min_samples_split': [2, 3, 4, 5, 6],
	'min_samples_leaf': [1, 2, 3, 4, 5, 6]

}

rfc = RandomForestClassifier(random_state=rs, n_jobs=-1)

search_cv = RandomizedSearchCV(rfc, params, cv=3, n_jobs=-1, random_state=rs, scoring='roc_auc')
search_cv.fit(features_train, target_train)

RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(n_jobs=-1,
                                                    random_state=250722),
                   n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 7, 10, 12, 13, 14,
                                                      16],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6],
                                        'min_samples_split': [2, 3, 4, 5, 6],
                                        'n_estimators': [30, 50, 70, 100]},
                   random_state=250722, scoring='roc_auc')

In [214]:
search_cv.best_params_

{'n_estimators': 70,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_depth': 13}

Получили неплохой ROC-AUC на кроссвалидации случайного леса

In [215]:
search_cv.best_score_

0.9286209851854874

Обучим модель с полученными гиперпараметрами

In [216]:
rfc = RandomForestClassifier(**search_cv.best_params_, random_state=rs, n_jobs=-1)
rfc.fit(features_train, target_train)

RandomForestClassifier(max_depth=13, n_estimators=70, n_jobs=-1,
                       random_state=250722)

Оценим модель

In [217]:
probabilities_valid = rfc.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

plot_roc_auc_curve(target_test, probabilities_one_valid)

In [218]:
confusion_matrix(target_test, rfc.predict(features_test))

array([[ 793,  248],
       [  37, 1076]], dtype=int64)

Ошибок второго рода гораздо больше, чем ошибок первого рода. Это значит, что заказчик будет чаще отправлять промо клиентам, которые и так не собирались уходить, но такой подход может повысить лояльность клиентов к компании.

# Вывдоы

В результате работы для телеком компании «Ниединогоразрыва.ком» было обучено две модели оттока клиентов CatboostClassifier и RandomForestClassifier.
Наилучшим образом себя показал RandomForestClassifier со следующими гиперпараметрами:

 - n_estimators - 70
 - min_samples_split - 2
 - min_samples_leaf - 1
 - max_depth - 13

Данная модель показала 0.9364 ROC-AUC.
На тестовой выборке, размером 2154 клиента модель показала
 - 37 ошибок первого рода
 - 248 ошибок второго рода

